# Community Detection with Leiden Algorithm

Community detection is one of the most fundamental tasks in network analysis. It involves partitioning the nodes of a graph into groups (communities) where nodes within the same community are more densely connected to each other than to nodes in other communities.

The **Leiden algorithm** is a popular community detection method that improves upon the classic Louvain algorithm. It guarantees well-connected communities and provides better quality partitions.

In this tutorial, we'll learn how to:
1. Generate a graph with known community structure
2. Apply the Leiden algorithm to detect communities
3. Tune parameters to control the number of communities
4. Visualize the results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.75)
sns.set_style("white")

from graspologic.partition import LeidenCluster
from graspologic.simulations import sbm
from graspologic.plot import heatmap

np.random.seed(42)

## Generate a Graph with Known Communities

We'll use a Stochastic Block Model (SBM) to generate a graph with a known community structure. This allows us to evaluate how well the Leiden algorithm recovers the true communities.

Our SBM will have 3 communities of 50 nodes each, with high within-community edge probability and low between-community edge probability.

In [ ]:
# Define community sizes
n_per_block = [50, 50, 50]

# Define block probability matrix
# High probability within communities (diagonal), low between communities (off-diagonal)
p = np.array([[0.5, 0.05, 0.02],
              [0.05, 0.5, 0.05],
              [0.02, 0.05, 0.5]])

# Generate the graph
G = sbm(n_per_block, p)

# Visualize the adjacency matrix
heatmap(G, title="SBM Adjacency Matrix (3 communities)")
plt.show()

print(f"Graph has {G.shape[0]} nodes")

The block structure is clearly visible in the adjacency matrix - the three diagonal blocks represent dense within-community connections.

## Apply Leiden Clustering

Now let's use the `LeidenCluster` class to detect communities in our graph. The Leiden algorithm will automatically determine the optimal number of communities based on modularity optimization.

In [ ]:
# Create and fit the Leiden clustering model
lc = LeidenCluster(resolution=1.0, random_seed=42)
communities = lc.fit_predict(G)

# Count communities found
n_communities = len(set(communities.values()))
print(f"Found {n_communities} communities")

# Show community assignments for first 10 nodes
print("\nCommunity assignments (first 10 nodes):")
for node in range(10):
    print(f"  Node {node}: Community {communities[node]}")

## Controlling the Number of Communities

Sometimes you may want to specify an exact number of communities. The `n_clusters` parameter allows you to do this - Leiden will try to find a partition with exactly that many communities.

In [ ]:
from graspologic.partition import leiden

# Detect exactly 4 communities
communities_4 = leiden(G, n_clusters=4, random_seed=42)

n_communities = len(set(communities_4.values()))
print(f"Requested 4 communities, found {n_communities}")

## Visualize Community Structure

Let's reorder the adjacency matrix by community assignment to better visualize the detected structure.

In [ ]:
# Get community labels as array (ordered by node index)
n_nodes = G.shape[0]
labels = np.array([communities[i] for i in range(n_nodes)])

# Sort nodes by community
sorted_indices = np.argsort(labels)
G_sorted = G[sorted_indices][:, sorted_indices]

# Plot sorted adjacency matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

heatmap(G, ax=axes[0], title="Original Adjacency Matrix")
heatmap(G_sorted, ax=axes[1], title="Sorted by Detected Communities")

plt.tight_layout()
plt.show()

## Evaluate Community Detection Quality

Since we generated the graph with known communities, we can evaluate how well Leiden recovered the true structure using the Adjusted Rand Index (ARI).

In [ ]:
from sklearn.metrics import adjusted_rand_score

# Create true labels (50 nodes per community)
true_labels = np.repeat([0, 1, 2], [50, 50, 50])

# Get predicted labels
predicted_labels = np.array([communities[i] for i in range(n_nodes)])

# Calculate ARI
ari = adjusted_rand_score(true_labels, predicted_labels)
print(f"Adjusted Rand Index: {ari:.4f}")
print("(1.0 = perfect match, 0.0 = random)")

## Summary

In this tutorial, we learned how to:
- Use `LeidenCluster` to detect communities in graphs
- Control the number of communities with `n_clusters`
- Visualize and evaluate community detection results

The Leiden algorithm is a powerful and efficient method for community detection that works well on a wide variety of network types.